In [14]:
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()  # Load environment variables from .env file

# Access keys and configurations
os.environ["LANGCHAIN_TRACING_V2"]=os.getenv('LANGCHAIN_TRACING_V2')
os.environ["LANGCHAIN_ENDPOINT"]=os.getenv('LANGCHAIN_ENDPOINT')
os.environ["LANGCHAIN_API_KEY"]=os.getenv('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"]=os.getenv('LANGCHAIN_PROJECT')
os.environ["ANTHROPIC_API_KEY"] = os.getenv('ANTHROPIC_API_KEY')
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # OpenAI API key
os.environ["NOTION_API_KEY"]=os.getenv('NOTION_API_KEY')
os.environ["LIBRARY_DATABASE_ID"]=os.getenv('LIBRARY_DATABASE_ID')


In [124]:
required_vars = [
    'LANGCHAIN_API_KEY',
    'ANTHROPIC_API_KEY',
    'NOTION_API_KEY',
    'LIBRARY_DATABASE_ID'
]

missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    raise EnvironmentError(f"Missing required environment variables: {', '.join(missing_vars)}")

In [ ]:
def get_current_date():
    # Get the current date
    current_date = datetime.now()
    # Format the date as YYYY-MM-DD
    formatted_date = current_date.strftime('%Y-%m-%d')
    return formatted_date

print(get_current_date())

In [8]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
# Initialize the language model
# llm = ChatOpenAI(model="gpt-4o-mini")

llm = ChatAnthropic(model="claude-3-sonnet-20240229")
llm.invoke("Hello, world!").content



In [ ]:
from notion_client import Client

# Initialize the Notion client with your integration token
notion = Client(auth=os.environ["NOTION_API_KEY"])

# Specify your database ID (the part of the URL after /<workspace>/)
database_id = os.environ["LIBRARY_DATABASE_ID"]

In [18]:
# Query the database
response = notion.databases.query(
    database_id=database_id
)

In [62]:
def extract_page_content(blocks):
    """Extract text content from page blocks."""
    content = []
    for block in blocks['results']:
        if block['type'] == 'paragraph':
            print('paragraph')
            try:
                text = block['paragraph']['rich_text']
                if text:
                    content.append(text[0]['plain_text'])
            except:
                continue
        elif block['type'] == 'heading_1':
            print('heading_1')
            try:
                text = block['heading_1']['rich_text']
                if text:
                    content.append(f"# {text[0]['plain_text']}")
            except:
                continue
        elif block['type'] == 'heading_2':
            print('heading_2')
            try:
                text = block['heading_2']['rich_text']
                if text:
                    content.append(f"## {text[0]['plain_text']}")
            except:
                continue
        elif block['type'] == 'bulleted_list_item':
            print('bulleted_list_item')
            try:
                text = block['bulleted_list_item']['rich_text']
                if text:
                    content.append(f"• {text[0]['plain_text']}")
            except:
                continue
    return '\n'.join(content)

In [ ]:
# Print out the results (summary, titles, etc.)
all_content = '<CONTEXT>'
id = 1
for page in response['results']:
    page_id = page['id']
    title = page['properties']['Name']['title'][0]['text']['content']
    summary = page['properties']['AI summary']['rich_text'][0]['text']['content']
    # print(id,title)
    
    page_content = notion.blocks.children.list(block_id=page_id)
    extracted_content = extract_page_content(page_content)
    
    content = '\n<Article Number: ' + str(id) + '>\n' + 'Title: ' + title + '\n' + 'Content: ' + extracted_content + '\n</Article Number: ' + str(id) + '>'
    all_content += content
    
    id += 1
all_content += '\n</CONTEXT>\n--------------------\n'

print(all_content)


In [ ]:
structured_prompt_newsletter = PromptTemplate(
    input_variables=["topic"],
    template=all_content + "<START USER PROMPT>\nProvide a weekly newsletter from the context provided. Use the context as a reference to what the user knows, but use outside sources to provide more information as needed.<END USER PROMPT>"
)

chain = structured_prompt_newsletter | llm
print(chain.invoke({"topic": "evals"}).content)

In [ ]:
# Create a one-shot example template
NEWSLETTER_EXAMPLE = """
Subject: AI & Tech Weekly Summary {date}

Welcome to this week's AI & Tech digest! Here's what's making waves:

Featured Story: The Evolution of Large Language Models
Last week's breakthrough in parameter-efficient training has opened new possibilities for smaller companies. Key highlights:
• 40% reduction in training costs
• Improved performance on specialized tasks
• New benchmarks for model efficiency

Industry Updates:
• Google announced their latest quantum computing milestone
• OpenAI released updates to their fine-tuning API
• Meta's PyTorch 2.0 shows promising performance gains

Must-Read Resources:
• New paper on efficient training methods [link]
• Updated documentation for PyTorch 2.0 [link]
• Comprehensive guide to quantum computing basics [link]

Join us next week for more updates!
-------------------
""".format(date=get_current_date())

newsletter_example_formatted = """<OUTPUT EXAMPLE>
{example}
</OUTPUT EXAMPLE>
""".format(example=NEWSLETTER_EXAMPLE)

newsletter_prompt = PromptTemplate(
    input_variables=["context", "today_date"],
    template= newsletter_example_formatted + """{context}

Generate today's newsletter that follows the output example format while incorporating the key points from the provided context. Make sure to have at least three bullet points in each section. Add relevant sections as needed, but maintain the professional and engaging tone.
Make sure to use today's date, {today_date}, in the subject line.
"""
)

# Create the chain
chain = newsletter_prompt | llm
# Test the chain
newsletter = chain.invoke({"context": all_content, "today_date": get_current_date()})
print(newsletter.content)